In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd

# Tentukan path folder tempat file PDF dan metadata.csv berada
folder_path = "/content/drive/MyDrive/Sertif Piagam GSS/"

# Membaca file CSV dari folder yang telah ditentukan
metadata_df = pd.read_csv(os.path.join(folder_path, 'metadata.csv'))

# Tentukan path ke folder tempat file PDF berada
pdf_folder_path = os.path.join(folder_path, "Peserta (271)")

# Dapatkan daftar file dalam folder
files = os.listdir(pdf_folder_path)

# Cek file PDF dan rename sesuai data dari metadata.csv
for index, row in metadata_df.iterrows():
    old_name = row['old_name']
    new_name = row['new_name']

    # Gabungkan path folder dan nama file
    old_file_path = os.path.join(pdf_folder_path, old_name)
    new_file_path = os.path.join(pdf_folder_path, new_name)

    # Cek apakah file lama ada di dalam folder
    if old_name in files and old_name.endswith(".pdf"):  # Pastikan hanya file .pdf
        try:
            # Rename file
            os.rename(old_file_path, new_file_path)
            print(f"File '{old_name}' berhasil diganti menjadi '{new_name}'")
        except Exception as e:
            print(f"Error mengganti nama file '{old_name}': {e}")
    else:
        print(f"File '{old_name}' tidak ditemukan atau bukan file PDF.")



In [ ]:
import os

def count_files(folder_path):
    """Counts the number of files in the specified folder."""
    return len(os.listdir(folder_path))

def total_files_count(folder_paths):
    """Counts the total number of files in all specified folders."""
    total_count = 0
    for folder in folder_paths:
        total_count += count_files(folder)
    return total_count

# Tentukan path folder yang ingin dihitung
folder_paths = [
    "/content/drive/MyDrive/Sertif Piagam GSS/Peserta (271)",
    "/content/drive/MyDrive/Sertif Piagam GSS/Grand Finalis (32)",
    "/content/drive/MyDrive/Sertif Piagam GSS/Piagam (39)"
]

# Menghitung dan menampilkan jumlah file untuk setiap folder
for folder in folder_paths:
    file_count = count_files(folder)
    print(f"Total number of files in '{folder}': {file_count}")

# Menghitung total jumlah file dari semua folder
total_file_count = total_files_count(folder_paths)

# Menampilkan total jumlah file
print(f"\nTotal number of files in all folders: {total_file_count}")


In [ ]:
import os

def count_files(folder_path):

  return len(os.listdir(folder_path))

folder_path = "/content/drive/MyDrive/Sertif Piagam GSS/Peserta (271)"

# Menghitung dan menampilkan jumlah file
file_count = count_files(folder_path)
print(f"\nTotal number of files in the folder: {file_count}")

# Dapatkan daftar file di folder
files = os.listdir(folder_path)

# Menampilkan daftar file beserta path-nya
for file in files:
    print(os.path.join(folder_path, file))

In [ ]:
!pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# Ganti dengan path ke file kredensial Anda
SERVICE_ACCOUNT_FILE = 'just-strength-438612-e8-e5a9e7327f4e.json'

# Scopes yang diperlukan
SCOPES = ['https://www.googleapis.com/auth/drive']

# Autentikasi menggunakan file kredensial
creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Membuat service
service = build('drive', 'v3', credentials=creds)


In [ ]:
# !pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import sys
import time

# Ganti dengan path ke file kredensial Anda
SERVICE_ACCOUNT_FILE = 'just-strength-438612-e8-e5a9e7327f4e.json'

# Scopes yang diperlukan
SCOPES = ['https://www.googleapis.com/auth/drive']

# Autentikasi menggunakan file kredensial
creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Membuat service
service = build('drive', 'v3', credentials=creds)

# Daftar file yang sudah Anda sebutkan
files = [
]

# Fungsi untuk menampilkan loading bar
def loading_bar(iteration, total, length=40, fill='█'):
    percent = (iteration / total) * 100
    filled_length = int(length * iteration // total)
    bar = fill * filled_length + '-' * (length - filled_length)
    sys.stdout.write(f'\r|{bar}| {percent:.2f}%')
    sys.stdout.flush()

# Mendapatkan ID file dan membuat link
print("Uploading files...\n")  # Pesan awal
file_links = []  # Menyimpan link di sini
for i, file_path in enumerate(files):
    # Mendapatkan nama file
    file_name = file_path.split("/")[-1]

    # Upload file ke Google Drive (jika belum diupload)
    file_metadata = {
        'name': file_name,
        'mimeType': 'application/pdf'
    }
    media = MediaFileUpload(file_path, mimetype='application/pdf')

    # Upload file
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    file_id = file.get('id')

    # Membuat permission untuk shareable link
    permission = {
        'type': 'anyone',
        'role': 'reader'
    }
    service.permissions().create(fileId=file_id, body=permission).execute()

    # Menyimpan link view-only berdasarkan ID file yang sebenarnya
    view_link = f"https://drive.google.com/file/d/{file_id}/view?usp=sharing"
    file_links.append(view_link)  # Menyimpan link ke list

    # Mengupdate loading bar di bawahnya
    loading_bar(i + 1, len(files))

# Setelah selesai, cetak semua link di bawah loading bar
print("\n\nUpload completed.")
print("Generated Links:")
for link in file_links:
    print(link)